## Multilayer Perceptron - MLP Network

**Autor:** Guilherme Cadori

**Data:** 17/03/2024


In [1]:
# Importando bibliotecas de trabalho
import numpy as np


In [2]:
# Gerando random seed
np.random.seed(123)

# Dados de treinamento
# Criando dados
# Criando x1 e x2 como uma matriz de inputs, na qual coluna 1 = x1 e coluna 2 = x2
X_train = np.array([[0, 1],
                    [0, 2],
                    [1, 1],
                    [1, 2],
                    [1, 3],
                    [2, 2],
                    [2, 3],
                    [3, 2],
                    [4, 1],
                    [4, 3],
                    [0, 3],
                    [2, 0],
                    [2, 1],
                    [3, 0],
                    [3, 1],
                    [3, 3],
                    [4, 0],
                    [4, 2],
                    [5, 0],
                    [5, 1],
                    [5, 2],
                    [5, 3]])

# Criando vetor de alvos
y_train = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])



In [3]:
# Ajustando formatos
m = X_train.shape[0]
X_train = np.transpose(X_train)
y_train = y_train.reshape((1, y_train.shape[0]))

print ('Forma de X: ' + str(X_train.shape))
print ('Forma de Y: ' + str(y_train.shape))
print (f"Exemplos: {m}")


Forma de X: (2, 22)
Forma de Y: (1, 22)
Exemplos: 22


In [4]:
# Gerando random seed para fins de repetibilidade
np.random.seed(123)

# Criando função sigmoid
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    
    return res


In [5]:
# Criando função de definição da estrutura da rede
def layer_sizes(X, Y):
    # Dimensões das camadas, considerando uma unica camada oculta
    n_x = X.shape[0]
    n_h = 1
    n_y = Y.shape[0]

    return (n_x, n_h, n_y)


In [6]:
# Testando função
(n_x, n_h, n_y) = layer_sizes(X=X_train, Y=y_train)
print("Input layer : n_x = " + str(n_x))
print("Hidden layer: n_h = " + str(n_h))
print("Output layer: n_y = " + str(n_y))


Input layer : n_x = 2
Hidden layer: n_h = 1
Output layer: n_y = 1


In [7]:
# Inicialização aleatória de parâmetros
def initialize_parameters(n_x, n_h, n_y):

    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    
    parameters = {"W1": W1,
                  "b1": b1}
    
    return parameters


In [8]:
# Testando função
parameters = initialize_parameters(n_x, n_h, n_y)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))



W1 = [[-0.01085631  0.00997345]]
b1 = [[0.]]


In [9]:
# Criando função de forward propagation
def forward_propagation(X, parameters):
    
    W1 = parameters['W1']
    b1 = parameters['b1']

    Z1 = W1 @ X + b1
    A1 = sigmoid(Z1)

    assert(A1.shape == (n_y, X.shape[1]))

    cache = {"Z1": Z1,
             "A1": A1}
    
    return A1, cache



In [10]:
# Conferindo resultados
A1, cache = forward_propagation(X_train, parameters)

print(A1)


[[0.50249334 0.50498656 0.49977929 0.50227264 0.50476587 0.49955857
  0.50205193 0.49684454 0.49163784 0.49662384 0.50747953 0.49457206
  0.49706524 0.49185849 0.49435137 0.49933786 0.4891454  0.49413069
  0.48643295 0.48892479 0.49141719 0.49391001]]


In [11]:
# Calculando custo
def compute_cost(A1, Y):
    
    # Quantidade de exemplos
    m = Y.shape[0]
    
    # Se Y for unidimensional: transformar em um array bidimensional para compatibilidade
    if Y.ndim == 1:
        Y = Y.reshape(1, -1)
        
    # Se A1 não estiver no formato (m, 1): ajustar formato
    if A1.shape[1] != 1:
        A1 = A1.reshape(-1, 1)

    # Calculando custo
    logloss = -Y * np.log(A1) - (1 - Y) * np.log(1 - A1)
    cost = np.sum(logloss) / m
    
    return cost


In [12]:
# Testando a função
print("Custo = " + str(compute_cost(A1, y_train)))


Custo = 332.4273404737903


In [13]:
# Criandon função para o backward propagation
def backward_propagation(parameters, cache, X, Y):
    
    # Capturando a dimensão de X
    m = X.shape[1]
    
    # Recuperando o valor de W1
    W1 = parameters["W1"]
    
    # Recuperando o valor de A1 do objeto cache criado anteriormente
    A1 = cache["A1"]
    
    # Reshape Y para ter a mesma forma de A1
    Y = Y.reshape(A1.shape)
    
    # Backward propagation: calculando derivadas parciais 
    dZ1 = A1 - Y
    dW1 = 1/m * np.dot(dZ1, X.T)
    db1 = 1/m * np.sum(dZ1, axis = 1, keepdims = True)
 
    grads = {"dW1": dW1,
             "db1": db1}
    
    return grads


In [14]:
# Testando a função
grads = backward_propagation(parameters, cache, X_train, y_train)

print("dW1 = " + str(grads["dW1"]))
print("db1 = " + str(grads["db1"]))


dW1 = [[2.36954845 0.36205361]]
db1 = [[0.49680182]]


In [15]:
# Função de apredendizado de parâmetros baseada em Gradiente Descendente
def update_parameters(parameters, grads, learning_rate=0.8):

    W1 = parameters['W1']
    b1 = parameters['b1']

    
    dW1 = grads['dW1']
    db1 = grads['db1']
   

    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1

    
    parameters = {"W1": W1,
                  "b1": b1}
    
    return parameters



In [16]:
# Testando função
parameters_updated = update_parameters(parameters, grads)

print("W1 updated = " + str(parameters_updated["W1"]))
print("b1 updated = " + str(parameters_updated["b1"]))


W1 updated = [[-1.90649507 -0.27966943]]
b1 updated = [[-0.39744145]]


In [17]:
# Criando função final de Multilayer Perceptron
def nn_model(X, Y, n_h, num_iterations=10, learning_rate=0.5, print_cost=False):

    # Dimensões de input e output
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    # Loop principal
    for i in range(0, num_iterations):
        
        # Forward propagation
        A1, cache = forward_propagation(X, parameters)

        # Função de custo
        cost = compute_cost(A1, Y)
        
        # Backpropagation
        grads = backward_propagation(parameters, cache, X, Y)
        
        # Aprendizado via Gradiente Descendente
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Custo a cada iteração
        if print_cost:
            print("Custo após iteração %i: %f" %(i, cost))
    
    # Custo final
    print("Custo final após iteração %i: %f" %(i+1, cost))
    
    return parameters


In [18]:
parameters = nn_model(X_train, y_train, n_h=1, 
                      num_iterations=110, 
                      learning_rate=0.01, 
                      print_cost=False)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))

W1 = parameters["W1"]
b1 = parameters["b1"]


Custo final após iteração 110: 67.116902
W1 = [[-1.46913471  0.12064306]]
b1 = [[-0.20516692]]


### Criando estimativsas

In [19]:
# Criando dados
# Criando x1 e x2 como uma matriz de inputs, na qual coluna 1 = x1 e coluna 2 = x2
X_test = np.array([[0  , 0  ],
                   [1  , 0  ],
                   [4.5, 0.5],
                   [3.5, 1.5],
                   [4  , 2.5],
                   [1.5, 1.5],
                   [2  , 0.5],
                   [2.5, 2.5]])

# Criando vetor de alvos
y_test = np.array([1, -1, -1, 1, 1, 1, -1, 1])


In [20]:
# Ajustando formatos
m = X_test.shape[0]
X_test = np.transpose(X_test)
y_test = y_test.reshape((1, y_test.shape[0]))

print ('Forma de X: ' + str(X_test.shape))
print ('Forma de Y: ' + str(y_test.shape))
print (f"Exemplos: {m}")


Forma de X: (2, 8)
Forma de Y: (1, 8)
Exemplos: 8


In [21]:
def predict(X, parameters):
    
    predictions = []
    
    A1, cache = forward_propagation(X, parameters)
    
    # Iterar sobre cada amostra
    for i in range(A1.shape[1]):
        # Verificar se o valor em A1 é maior que 0.5
        if A1[0, i] > 0.5:
            predictions.append(1)
        else:
            predictions.append(-1)
    
    return predictions


In [22]:
# Estimando valores
y_pred = predict(X_test, parameters)

# Calculando erro
erro_percent = np.count_nonzero(y_pred != y_test) / m * 100

# Comparando resultados
print('Estimativas finalizadas')
print(f"Erro Percentual: {erro_percent}%")
print(f"Estimativas    : {y_pred}")
print(f"Valores Alvo   : {y_test}")



Estimativas finalizadas
Erro Percentual: 62.5%
Estimativas    : [-1, -1, -1, -1, -1, -1, -1, -1]
Valores Alvo   : [[ 1 -1 -1  1  1  1 -1  1]]


**Fim**